In [70]:
from functools import partial
from sqlite3 import connect

import astroplan as ap
import astropy.units as u
from icecream import ic, install

import astropaul.targetlistcreator as tlc
import astropaul.html as html
import astropaul.phase as ph
import astropaul.priority as pr

install() # icecream will now work in imported packages

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [171]:
conn = connect("../../../../Data/astropaul.db")

session = tlc.ObservingSession(ap.Observer.at_site("APO"))
session.add_full_day("2025-06-07")
session.add_full_day("2025-06-08")
session.add_full_day("2025-06-10")
session.add_full_day("2025-06-11")
session.add_half_day("2025-06-12", first_half=True)

synthetic_phase_percent = 0.01

phase_event_defs = [
    ph.PhaseEventDef("Mid Eclipse", ph.calc_mid_eclipse),
    ph.PhaseEventDef("Eclipse", ph.calc_mid_eclipse),
    ph.PhaseEventDef(
        "Not in Eclipse", partial(ph.calc_time_of_gress, synthetic_phase_percent=synthetic_phase_percent, ingress=False)
    ),
    ph.PhaseEventDef("Eclipse", partial(ph.calc_time_of_gress, synthetic_phase_percent=synthetic_phase_percent, ingress=True)),
]

min_altitude = 35 * u.deg
max_magnitude = 13

name = "APO Observing List 2025-06-07 to -12"
creator = tlc.TargetListCreator(name=name, connection=conn)
creator.steps = [
    tlc.add_targets,
    partial(tlc.filter_targets, criteria=lambda df: (df["Target Type"].isin(["QuadEB", "SextEB"]))),
    # partial(tlc.filter_targets, criteria=lambda df: (df["Target Name"] == "TIC 123098844")),
    tlc.ancillary_data_from_tess,
    partial(
        tlc.add_observability,
        observing_session=session,
        calc_moon_distance=True,
        observability_threshold=(min_altitude, 80 * u.deg),
    ),
    partial(tlc.filter_targets, criteria=lambda df: (df["Observable Any Night"])),
    partial(tlc.filter_targets, criteria=lambda df: (df["Vmag"] < max_magnitude)),
    partial(tlc.add_database_table, table_name="ephemerides"),
    # partial(tlc.add_phase_events, observing_session=session, phase_event_defs=phase_event_defs),
    partial(
        tlc.add_phase_events,
        observing_session=session,
        phase_event_defs=phase_event_defs,
        event_types=["Mid Eclipse", "Eclipse", "Not in Eclipse"],
        # event_types=["Ingress", "Mid Eclipse", "Egress", "In Eclipse"],
    ),
    partial(tlc.add_database_table, table_name="dssi_observations"),
    partial(
        tlc.add_system_configuration, table_name="DSSI Observations", time_column="Mid JD", eclipse_table="SIDE Observations"
    ),
    partial(tlc.add_database_table, table_name="speckle_detections"),
    # partial(tlc.add_side_status, phase_event_defs=phase_event_defs),
]
tl = creator.calculate(verbose=False)
# html.render_observing_pages(tl, None, {}, "html")

print(tl.summarize())
# tl.other_lists["SIDE Observations"]

Name: APO Observing List 2025-06-07 to -12
Criteria
    lambda df: (df["Target Type"].isin(["QuadEB", "SextEB"]))),
    Observability threshold: (<Quantity 35. deg>, <Quantity 80. deg>)
    AltitudeConstraint: {'min': np.float64(35.0), 'max': np.float64(80.0), 'boolean_constraint': True}
    lambda df: (df["Observable Any Night"])),
    lambda df: (df["Vmag"] < 13)),
35 targets:
      35 QuadEB
Column Count (primary, secondary):
    Target: (3, 4)
    TESS Data: (2, 4)
    Observable: (5, 20)
    Count: (3, 0)
Associated tables:
       5 rows,  2 columns: Lunar Phases
     117 rows,  7 columns: Ephemerides
     916 rows, 10 columns: Phase Events
      97 rows, 12 columns: DSSI Observations
      42 rows,  6 columns: SIDE Observations
      54 rows, 11 columns: Speckle Detections



In [172]:
altitude_categories = [
    ((-90, min_altitude.value), 0),
    ((min_altitude.value, 90), 1),
]

pl = pr.PriorityList(tl, session, interval=30 * u.min)
pr.calculate_altitude_priority(pl, altitude_categories=altitude_categories)
# pr.prioritize_phase_sequence(pl, ["In Eclipse", "Mid Eclipse", "In Eclipse"], "Eclipse", True, True, True)
pr.prioritize_phase_sequence(pl, ["Eclipse"], "Eclipse", True, True, True)
pr.prioritize_side_observation(pl, side_state="Eclipse")
pr.calculate_overall_priority(pl)
pr.aggregate_target_priorities(pl, skip_column_threshold=0.1)
pl.categorize_priorities(bins=[0.00, 0.20, 0.40, 0.6, 1.00], labels=["", "*", "* *", "* * *"])

html.render_observing_pages(tl, pl, {}, "../../../../Observing Files/APO Observing 2025-06-07 to -12 Improved")
# pl.numerical_priorities[0]